In [11]:
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
min_val = 0
max_val = 100

opcash_to_capex = pd.read_csv('opcash_to_capex_tech.csv').drop(columns=['Unnamed: 0'])
opcash_to_capex = opcash_to_capex[(opcash_to_capex['Operating cash flow to capex'] <= max_val)&
                                  (opcash_to_capex['Operating cash flow to capex'] >= min_val) &
                                  ~(opcash_to_capex['capex'] == 0)]
opcash_to_capex.sort_values(by='Operating cash flow to capex', ascending=True)
annual_returns = pd.read_csv('annual_returns.csv').rename(columns={'Date':'date', 'Close': 'close(%)'})
daily_returns = pd.read_csv('daily_close.csv').rename(columns={'Date':'date', 'Close': 'close'})
daily_returns

,date,close,ticker
0,1994-01-10 00:00:00-05:00,0.248125,AAPL
1,1994-01-11 00:00:00-05:00,0.235212,AAPL
2,1994-01-12 00:00:00-05:00,0.225065,AAPL
3,1994-01-13 00:00:00-05:00,0.225988,AAPL
4,1994-01-14 00:00:00-05:00,0.228755,AAPL
...,...,...,...
1315928,2024-01-03 00:00:00-05:00,10.840000,WAVS
1315929,2024-01-04 00:00:00-05:00,10.860000,WAVS
1315930,2024-01-05 00:00:00-05:00,10.890000,WAVS
1315931,2024-01-08 00:00:00-05:00,10.890000,WAVS


In [13]:
annual_returns['date'] = pd.to_datetime(annual_returns['date']).dt.year
daily_returns['date'] = pd.to_datetime(daily_returns['date'], utc=True)
daily_returns['date'] = daily_returns['date'].dt.year
daily_returns


,date,close,ticker
0,1994,0.248125,AAPL
1,1994,0.235212,AAPL
2,1994,0.225065,AAPL
3,1994,0.225988,AAPL
4,1994,0.228755,AAPL
...,...,...,...
1315928,2024,10.840000,WAVS
1315929,2024,10.860000,WAVS
1315930,2024,10.890000,WAVS
1315931,2024,10.890000,WAVS


In [14]:
opcash_to_capex = opcash_to_capex[opcash_to_capex.groupby('ticker')['ticker'].transform('count') >= 10]
opcash_to_capex

,date,Operating cash flow,ticker,capex,Operating cash flow to capex
0,2023,1.105430e+11,AAPL,-1.095900e+10,9.913789
1,2022,1.221510e+11,AAPL,-1.070800e+10,8.766199
2,2021,1.040380e+11,AAPL,-1.108500e+10,10.654761
3,2020,8.067400e+10,AAPL,-7.309000e+09,9.059920
4,2019,6.939100e+10,AAPL,-1.049500e+10,15.124440
...,...,...,...,...,...
3844,2015,2.864500e+07,CAMP,-7.437000e+06,25.962646
3845,2014,2.281600e+07,CAMP,-2.133000e+06,9.348703
3846,2013,1.659700e+07,CAMP,-1.852000e+06,11.158643
3847,2012,1.243200e+07,CAMP,-1.076000e+06,8.655084


In [15]:
opcash_to_capex['rank'] = opcash_to_capex.groupby('date')['Operating cash flow to capex'].rank(ascending=False)
pd.reset_option('display.max_rows', None)
opcash_to_capex.sort_values(by='rank')

,date,Operating cash flow,ticker,capex,Operating cash flow to capex,rank
2366,2006,-3.819970e+07,PLUS,-34505417.0,90.329029,1.0
3397,1998,6.283000e+06,RELL,-4116000.0,65.510107,1.0
1377,2009,2.617250e+08,AMKR,-218177000.0,83.361162,1.0
1370,2016,7.294020e+08,AMKR,-650038000.0,89.119306,1.0
2703,2002,4.953000e+06,SPNS,-3743000.0,75.570361,1.0
...,...,...,...,...,...,...
3658,2021,1.525200e+07,GSIT,-203000.0,1.330973,116.0
3117,2017,-5.223500e+07,CERS,-353000.0,0.675792,116.0
764,2021,1.203900e+09,CHKP,-15900000.0,1.320708,117.0
2845,2015,6.905500e+07,ERII,-572000.0,0.828325,117.0


In [16]:
opcash_to_capex['quintile'] = pd.qcut(opcash_to_capex['rank'], q=5, labels=False, duplicates='drop') + 1


In [17]:
quintile_portfolios = {}

for q in range(1, 6):
    quintile_portfolios[q] = opcash_to_capex[opcash_to_capex['quintile'] == q].copy()


In [18]:
opcash_to_capex[(opcash_to_capex['date']==2010) &
                (opcash_to_capex['quintile']==1)].sort_values(by='rank')

,date,Operating cash flow,ticker,capex,Operating cash flow to capex,rank,quintile
932,2010,427410000.0,JBL,-398425000.0,93.218455,1.0,1
3743,2010,3697000.0,KTCC,-3378000.0,91.371382,2.0,1
1993,2010,121445000.0,TSEM,-106710000.0,87.866936,3.0,1
2484,2010,4838000.0,MXL,-4235000.0,87.536172,4.0,1
1376,2010,542595000.0,AMKR,-445669000.0,82.136584,5.0,1
2726,2010,-4857000.0,UCTT,-3817000.0,78.587606,6.0,1
2455,2010,95919000.0,PLAB,-71381000.0,74.417999,7.0,1
440,2010,359000000.0,NXPI,-265000000.0,73.816156,8.0,1
2727,2010,1091000.0,UCTT,-738000.0,67.644363,9.0,1
3540,2010,11009000.0,AXTI,-6386000.0,58.007085,10.0,1


In [19]:
returns = opcash_to_capex.merge(annual_returns, how='inner', on=['ticker', 'date'])
avg_returns = returns.groupby(['date', 'quintile'])['close(%)'].mean()
# returns_roll_3 = returns.groupby(['quintile', 'date'])['close(%)'].rolling(window=3).mean().reset_index()
# returns_roll_3
returns

,date,Operating cash flow,ticker,capex,Operating cash flow to capex,rank,quintile,close(%)
0,2023,1.105430e+11,AAPL,-1.095900e+10,9.913789,34.0,2,0.353998
1,2022,1.221510e+11,AAPL,-1.070800e+10,8.766199,81.0,4,-0.392748
2,2021,1.040380e+11,AAPL,-1.108500e+10,10.654761,76.0,4,0.275682
3,2020,8.067400e+10,AAPL,-7.309000e+09,9.059920,81.0,4,0.438959
4,2019,6.939100e+10,AAPL,-1.049500e+10,15.124440,66.0,4,0.470177
...,...,...,...,...,...,...,...,...
1630,2015,2.864500e+07,CAMP,-7.437000e+06,25.962646,45.0,3,0.079277
1631,2014,2.281600e+07,CAMP,-2.133000e+06,9.348703,91.0,5,-0.480874
1632,2013,1.659700e+07,CAMP,-1.852000e+06,11.158643,85.0,4,0.691813
1633,2012,1.243200e+07,CAMP,-1.076000e+06,8.655084,93.0,5,0.486779


In [23]:
excess_returns = pd.DataFrame(avg_returns-returns['close(%)'].mean()).rename(columns={'close(%)':'excess returns'})
excess_returns.merge(avg_returns, on='quintile')#.to_csv('excess_returns.csv')
# excess_returns

,excess returns,close(%)
quintile,,
1,-0.111885,-0.115615
1,-0.111885,0.323522
1,-0.111885,-7.475482
1,-0.111885,-0.613106
1,-0.111885,-1.783651
...,...,...
5,-0.342497,-0.366736
5,-0.342497,0.120989
5,-0.342497,0.280255


In [50]:
avg_returns = returns.groupby(['date', 'quintile'])['close(%)'].mean()
# pd.reset_option('display.max_rows',)
pd.DataFrame(avg_returns)

close(%)
date quintile          
1998 1        -0.115615
1999 1         0.323522
2000 1        -7.475482
2001 1        -0.613106
2002 1        -1.783651
...                 ...
2022 4        -0.631526
     5        -0.346227
2023 1        -0.748423
     2         0.208146
     3         0.143868

[84 rows x 1 columns]

In [14]:
returns['close(%)'].mean()

-0.003730182638564316

## CAGR

In [15]:
def cagr(beginning_value, ending_value, years):
    cagr_val = (ending_value / beginning_value) ** (1 / years) - 1
    return cagr_val

### Calculating annual quintile sum

In [27]:
# daily_returns.groupby(['date', 'ticker'])['close'].mean().reset_index()
returns

,date,Operating cash flow,ticker,capex,Operating cash flow to capex,rank,quintile,close(%)
0,2023,1.105430e+11,AAPL,-1.095900e+10,9.913789,34.0,2,0.353998
1,2022,1.221510e+11,AAPL,-1.070800e+10,8.766199,81.0,4,-0.392748
2,2021,1.040380e+11,AAPL,-1.108500e+10,10.654761,76.0,4,0.275682
3,2020,8.067400e+10,AAPL,-7.309000e+09,9.059920,81.0,4,0.438959
4,2019,6.939100e+10,AAPL,-1.049500e+10,15.124440,66.0,4,0.470177
...,...,...,...,...,...,...,...,...
1630,2015,2.864500e+07,CAMP,-7.437000e+06,25.962646,45.0,3,0.079277
1631,2014,2.281600e+07,CAMP,-2.133000e+06,9.348703,91.0,5,-0.480874
1632,2013,1.659700e+07,CAMP,-1.852000e+06,11.158643,85.0,4,0.691813
1633,2012,1.243200e+07,CAMP,-1.076000e+06,8.655084,93.0,5,0.486779


In [54]:
years = 2023-2019
for i in range(1,6):
    print (i)
    start_val = yearly_avg_returns[yearly_avg_returns['quintile'] == i].reset_index().loc[0, 'close']
    end_val = yearly_avg_returns[yearly_avg_returns['quintile'] == i].reset_index().iloc[-1]['close']
    print(cagr(start_val, end_val, years))

1
nan
2
-0.16650206648662003
3
-0.21810373399267768
4
nan
5
nan


/var/folders/_p/y56kr6vs1fz7ycsbxhyqwwwc0000gn/T/ipykernel_65913/3499885254.py:2: RuntimeWarning: invalid value encountered in scalar power
  cagr_val = (ending_value / beginning_value) ** (1 / years) - 1
/var/folders/_p/y56kr6vs1fz7ycsbxhyqwwwc0000gn/T/ipykernel_65913/3499885254.py:2: RuntimeWarning: invalid value encountered in scalar power
  cagr_val = (ending_value / beginning_value) ** (1 / years) - 1
/var/folders/_p/y56kr6vs1fz7ycsbxhyqwwwc0000gn/T/ipykernel_65913/3499885254.py:2: RuntimeWarning: invalid value encountered in scalar power
  cagr_val = (ending_value / beginning_value) ** (1 / years) - 1


In [56]:
start_val = yearly_avg_returns[yearly_avg_returns['quintile'] == 1].reset_index().loc[0, 'close']
end_val = yearly_avg_returns[yearly_avg_returns['quintile'] == 1].reset_index().iloc[-1]['close']
print(cagr(start_val, end_val, years))
print(start_val, end_val, years)

nan
0.3891249488755727 -0.7484232400287456 4


/var/folders/_p/y56kr6vs1fz7ycsbxhyqwwwc0000gn/T/ipykernel_65913/3499885254.py:2: RuntimeWarning: invalid value encountered in scalar power
  cagr_val = (ending_value / beginning_value) ** (1 / years) - 1


$$CAGR = \left(1 + \frac{{\text{{Annual Change in Percent}}}}{{100}}\right)^{\frac{1}{{\text{{Number of Years}}}}} - 1$$

In [57]:
yearly_avg_returns[yearly_avg_returns['quintile'] == 1]

,date,quintile,close
12,2009,1,0.389125
16,2010,1,0.160521
21,2011,1,-1.050471
26,2012,1,0.086162
31,2013,1,0.292581
36,2014,1,0.052632
41,2015,1,-0.315685
46,2016,1,0.160418
51,2017,1,-0.023562
56,2018,1,-0.436766
